<a href="https://colab.research.google.com/github/AlexShashkov/laguerre_polynomials/blob/main/SPA15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math as m
from numpy import longcomplex as lc
import numpy.polynomial.polynomial as P

In [2]:
# глобальные переменные
# погрешности , по сути это костыли :)
prec = 10
eps = np.float64(1) / np.float64(10**prec)
eps1 = np.float64(1) / np.float64(10**(prec+1))
eps4 = np.float64(1) / np.float64(10**(2*prec))

In [3]:
# глобальная
ln10n = np.log(10)

#Bernoulid

In [4]:
# Done
def bernoullid(tp, tca, steps, tdeg, flag_min):
  k0 = next((i for i, x in enumerate((tca)) if x), None)+1
  print(k0, tca[k0])
  kden = -tca[0] / (k0*tca[k0]);
  kia = np.array([k0 for _ in range(steps+1)])
  tqa = np.array([tdeg*kden for _ in range(steps+1)])
  asa = np.array([0.0 + 0.0j for _ in range(tdeg+1)])
  qsa = np.array([0.0 + 0.0j for _ in range(tdeg+1)])
  print('Test', asa, tdeg)
  for n in range(steps+1):
    kn1 = kia[n-1]
    if kn1 == n:
      k = n
      be = max(0,n-tdeg)
      while True:
        k+=1
        se = min(k,tdeg)
        for t in range(k-n+1, se):
          if tqa[k-t] != 0.0:
            asa[t] = tca[t]
          else:
            asa[t] = 0.0
          for s in range(k-t-be, n-1-be):
            if s+be == 0.0:
              qsa[0] = k*kden
            else:
              if tqa[s+be] != 0.0:
                qsa[s] = tqa[s+be]
              else:
                qsa[s] = 1.0
        if se <= k-n:
          pn = 0.0
        else:
          pn = asa[se]*qsa[k-se-be]
        for j in range(se, k-n+2, -1):
          pn = (pn + asa[j-1]) * qsa[k-j-be+1]
        pn = pn + tca[k-n]
        if pn != 0.0 or k == tdeg:
          break
      kia[n] = k
      if pn == 0.0:
        return [], tqa, kia
      tqa[n] = -tca[0]/pn
    else:
      kia[n] = kn1
      tqa[n] = 0.0
  terr1 = err(tp, tqa[steps])[0]
  print('done')
  if terr1 < np.inf:
    if terr1 < eps:
      return [tqa[steps], terr1], tqa, kia
    else:
      if flag_min:
        '''
        if terr1 < lam:
          lam = terr1
          ats = tshift
          ass = sshift
        '''
        tph = tp
        stem = steps
        sqa = tqa
        kma = kia
      return [], tqa, kia




#Deflate

In [5]:
#TO DO
def deflate(rootlist, tp, u, bz1, tdeg, flag_suc, flag_fin):
  root = u+bz1
  rootlist, flag_fin, flag_lag = finroot(tp, rootlist, root, flag_suc, flag_fin)
  if flag_suc:
    tp1u = np.where(z==u, tp, z) #  replaces all instances of z in tp with u.
    tp1 = np.divide(tp1, z - u)
    coeff = np.polyfit(range(len(tp1)), tp1, deg=len(tp1)-1)
    tcoeff = coeff[0]  # The coefficient of the highest degree term !!!!!!!!!!!!!!!!!!!!!
    tp1 = -tp1 / tcoeff
    tp1.sort()
    tdeg = tdeg - 1
    if isinstance(root, np.complex) and tdeg > 2:
      u1 = np.conj(u + bz1) - bz1
      tcas = np.zeros(stepss + 2)
      tps = np.zeros((stepss + 2, len(tp1)))
      tps[0] = tp1
      for j in range(1, stepss + 2):
          tps[j] = np.gradient(tps[j - 1])
      for i in range(corsteps):
          for j in range(stepss + 2):
              tcas[j] = np.real(np.where(z == u1, tps[j], z)) / np.math.factorial(j)
          if np.real(tcas[0])**2 + np.imag(tcas[0])**2 < eps4:
              if np.real(tcas[1])**2 + np.imag(tcas[1])**2 > eps4:
                  u2 = u1 - tcas[0] / tcas[1]
                  terr = err(tp1, u2)[0]
                  if terr < inf:
                      if terr < eps:
                          finroot(u2 + bz1)
                          tp1 = np.divide(tp1, z - u2)
                          coeff = np.polyfit(range(len(tp1)), tp1, deg=len(tp1)-1)
                          tcoeff = coeff[0]  # The coefficient of the highest degree term !!!!!!!!!!!!
                          tp1 = -tp1 / tcoeff
                          tdeg = tdeg - 1
                          break

#Shift

In [6]:
#Done
def shift(u, tp1):
    t1 = tp1
    tp = []
    tp.append(np.polyval(tp1, u))
    _tp = np.polyder(t1)
    den = 1.0
    print('tp1', tp1)
    print('_tp', _tp)
    print('u', u)
    while len(_tp) > 0:
      tp.append(np.polyval(_tp, u)/den)
      den = den*(den+1.0)
      _tp = np.polyder(_tp)
      print('_tp', _tp)
      print('el', tp[-1])
      print('den', den)
    tp = list(reversed(tp))
    if tp[0] != 0:
      for el in tp:
        el /= tp[0]
    return tp



#Doublecos

In [7]:
#Done
def doublecos(cosa, noco):
  cosah = cosa
  d1 = noco // 4
  cosa = np.array([2.0 for _ in range(2*noco)])
  cosa[0] = np.sqrt(cosah[0] + 2)  # evalf в Maple это float в Python
  cosa[noco - 1] = -cosa[0]
  cosa[noco + 1] = -cosa[0]
  cosa[2 * noco - 1] = cosa[0]
  for i in range(noco-1):
    cosa[2*i] = cosah[i]
  for i in range(d1-1):
    j = 2*i+1
    cosa[j] = cosa[0]*cosa[j-1]-cosa[j-2]
    cosa[noco-j] = -cosa[j]
    cosa[noco+j] = -cosa[j]
    cosa[2*noco-j] = cosa[j]
  noco = 2*noco
  return cosa, noco



#Minima

In [8]:
#TO DO
def minima(cosa, ads, noco, rad, tdeg, tca):
  poa = np.array([[1.0, rad] for _ in range(tdeg + 1)])
  pka = [i for i in range(tdeg+1)]
  qka = [i for i in range(tdeg+1)]
  for i in range(1, tdeg+1):
    print(poa[i], rad)
    poa[i] = rad*poa[i-1]
  poa2 = poa[1]
  for k in range(tdeg+1):
    ph = 0
    for j in range(tdeg-k, 1, -1):
      ph = ph*poa2 + tca[j+k].conjugate()*tca[j]
    ph = poa[k]*(ph*poa2 + tca[k].conjugate()*tca[0])
    pka[k] = k*ph.real
    qka = k*ph.imag
  subex = max(5, np.ceil(np.log2(10.0 * rad * np.sqrt(tdeg)))) + ads
  subdiv = 3*np.power(2, subex)
  d3 = 3 * (subdiv // 4)  # iquo в Maple это целочисленное деление в Python
  noex = m.floor(np.log2(noco / 3) + 0.5) #Функция floor также из модуля math используется для округления числа вниз до ближайшего целого
  coh = np.zeros(subdiv)
  if subex >= noex:
    for _ in range(subex-noex):
      cosa, noco = doublecos(cosa, noco)
    coh =cosa
  else:
    fa = noco/subdiv
    for kk in range(subdiv-1):
      coh[kk] = cosa[fa*kk]
  indlist = []
  indlist2 = []
  si1 = 2.0 * sum(qka)
  if si1 > 1e-8:
      si1 = 1e-8
  elif si1 < -1e-8:
      si1 = -1e-8
  sil = [[0, si1]]
  si0 = si1
  for ij in range(subdiv):
    si2 = si1
    si1 = sum(qka[jj]*coh[jj*ij % subdiv] - pka[jj]*coh[(d3+jj*ij) % subdiv] for jj in range(tdeg+1))
    if si1 > 1e-8:
      si1 = 1e-8
    elif si1 < -1e-8:
      si1 = -1e-8
    sil.append([ij, si1])
    if (si2 <= 0) and (si1 > 0):
      indlist.append(ij-1)
      indlist2.extend([ij-1, ij])
  if (si1 <=0) and si0 > 0:
    indlist.append(subdiv-1)
    indlist2.extend([subdiv-1, 0])
  len = len(indlist)
  len2 = len(indlist2)
  if flag_once:
      flag_once = False

#Minrad

In [9]:
#Ошибка в коде
def minrad(tp, tqa, tdeg, steps, kia, flag_once1):
  if flag_once1:
    flag_once1 = False
    print('tp', tp)
    rl = np.roots(np.array(tp).astype('complex128'))
    print('rl', rl)
    rmin = np.inf
    '''
    for jl in range(len(rl)):
        if abs(rl[jl].real) < rmin:
            rmin = abs(rl[jl])
    '''
    print('tqa', tqa)
    rmin = np.round(rl[-1].conjugate())
    print('rmin', rmin) #найти минимальное
    rm3 = np.power(5.0, 1.0/16.0)*rmin
    mili = []
    mico = []
    mic3 = []
    tr = tqa[0]
    tq = tr.real**2 + tr.imag**2
    print('ch', kia)
    ch = kia[0]
    km = ch
    tm = tq
    while ch <= steps:
      tr = tqa[ch]
      tq *= (tr.real**2 + tr.imag**2)
      ch = kia[ch]
      km = ch
      tm = tq
      mili.append([km, np.power(tm, 1.0/(2.0*km))])
      mico.append([km, rmin])
      mic3.append([km, rm3])
      perp3 = [[16*tdeg, rmin], [16*tdeg, rm3]]
  else:
    tr = tqa[0]
    tq = tr.real**2 + tr.imag**2
    ch = kia[0]
    km = ch
    tm = tq
    while ch <= steps:
      tr = tqa[ch]
      tq *= (tr.real**2 + tr.imag**2)
      ch = kia[ch]
      if tq**km < tm**ch:
        km = ch
        tm = tq
  print('tm', tm, km)
  rm = np.power(tm, 1.0/(2.0*km))
  return rmin, flag_once1

#Laguerre

In [10]:
#Done
def laguerre(q, x):
  dq = np.polyder(q)
  v0 = np.polyval(q, x)
  v1 = np.polyval(dq, x)
  print('q', q)
  print('dq', dq)
  if v1 != 0.0:
    vh = v0 / v1
    if (abs(vh) < eps1):
      return vh
  if v1 == 0.0:
    return np.inf
  else:
    if v0 == 0.0:
      return 0.0
    else:
      return v0 / v1


#Err

In [11]:
#Done
def err(q, x):
  la = laguerre(q, x)
  flag_lag = True
  ala = (len(q)-1)*abs(la)
  if ala < np.inf:
    return [ala, flag_lag]
  else:
    return [np.inf, flag_lag]


#Finroot

In [12]:
#Done
# принимает на вход p - массив коэф, rootlist - корни, ro - число, flag_suc - какой-то flag (хз что он именно проверяет)
def finroot(p, rootlist, ro, flag_suc, flag_fin):
  #nera,lade,pof,bof - локальные переменные
  nera = err(p, ro);
  if nera[0] < np.inf:
    bof = nera[0] < eps1 #проверяем, меньше ли первый элемент массива nera чем eps1, и присваеваем переменной bof значение true или false.
  else:
    bof = False
  if flag_suc:
    #lade = np.round(ladi, 2)
    pof = ro
    rootlist.append(pof)
    #miro += 1
    if not bof:
      flag_fin = False
  return rootlist, flag_fin, nera[1]

#degtwo

In [13]:
#Done
def degtwo(bz, tp, tdeg, rootlist, flag_fin):
  flag_suc = True;
  flag_f = flag_fin
  a = tp[0]
  b = -0.5*tp[1]
  c = tp[2]
  d = np.sqrt(b**2 - c)
  a = b + d
  b = b - d
  if abs(a) > abs(b):
    rootlist, flag_f, flag_lag = finroot(tp, rootlist, bz + a, flag_suc, flag_fin)
    rootlist, flag_f, flag_lag = finroot(tp, rootlist, bz + c/a, flag_suc, flag_fin)
  if abs(a) == abs(b):
    rootlist, flag_f, flag_lag = finroot(tp, rootlist, bz + a, flag_suc, flag_fin)
    rootlist, flag_f, flag_lag = finroot(tp, rootlist, bz + b, flag_suc, flag_fin)
  if abs(a)<abs(b):
    rootlist, flag_f, flag_lag = finroot(tp, rootlist, bz + b, flag_suc, flag_fin)
    rootlist, flag_f, flag_lag = finroot(tp, rootlist, bz + c/b, flag_suc, flag_fin)
  tdeg -= 2
  return rootlist, tdeg, flag_f, flag_lag

#Trial

In [14]:
#Done
def trial(rootlist, cl, tp, tp1, tdeg, bz, bz1, stepfactor, stepmin, flag_suc,flag_fin, flag_inc, flag_min, flag_once1):
  print('trial_tp', tp)
  if (tp[-1].real**2 + tp[-1].imag**2) < eps4:
    if (tp[-2].real**2 + tp[-2].imag**2) > eps4:
      qco = -tp[-1]/tp[-2]
      ladi = err(tp, qco)[0]
      if ladi < np.inf and ladi < eps:
        if cl == 2:
          terr = err(tp1, qco + bz - bz1)[0]
          if terr < np.inf and terr >=eps:
            flag_inc = True
            print('trial_return1')
            return bz, bz1, flag_suc, flag_inc, flag_once1, None, None
        flag_suc = True
        print('TO DO deflate')
        deflate(rootlist, tp, qco + bz - bz1, bz1, tdeg, flag_suc, flag_fin)
        print('trial_return2')
        return bz, bz1, flag_suc, flag_inc, flag_once1, None, None
  print('tp', tp)
  # Assuming tp is a numpy array
  diam = 1.0 + (np.linalg.norm([el - tp[0] for el in tp], np.inf))/abs(tp[0])
  if cl == 1:
    steps = np.ceil(stepfactor *(np.log(diam*tdeg) + ln10n*(prec+1)))
  elif cl == 2 and flag_min:
    steps = np.ceil(stepmin*(np.log(diam*tdeg) + ln10n*(prec+1)))
  else:
    steps = 16 * tdeg
  tca = list(reversed(tp))
  steps = int(steps)
  res2, tqa, kia = bernoullid(tp, tca, steps, tdeg, flag_min)
  print('kia', kia, kia[0])
  if flag_inc:
    print('trial_return3')
    return bz, bz1, flag_suc, flag_inc, flag_once1, tca, None
  if len(res2) == 2:
    ladi = res2[1]
    flag_suc = True
    print('TO DO deflate')
    deflate(res2[0]+bz-bz1)
  elif len(res2) == 0:
    if cl == 1 or cl == 3:
      rad, flag_once1 = minrad(tp, tqa, tdeg, steps, kia, flag_once1)
    elif cl == 2 and flag_min:
      tqm = tqa
      kim = kia
  print('trial_return4')
  print('rad?', rad)
  return bz, bz1, flag_suc, flag_inc, flag_once1, tca, rad


#Run

In [15]:
def run(cosa, ads, noco, bz, bzre, tp, tp1, tdeg, rootlist,stepfactor, stepmin, flag_fin, flag_once1):
  flag_lag = True
  if tdeg < 3:
    return degtwo(bz, tp, tdeg, rootlist, flag_fin)
  else:
    flag_suc= False
    flag_inc = True
    flag_min = False
    bz = bzre
    while flag_inc:
      flag_inc = False
      tp1 = tp
      bz1 = bz
      bz, bz1, flag_suc, flag_inc, flag_once1, tca, rad = trial(rootlist, 1, tp, tp1, tdeg, bz, bz1, stepfactor,stepmin, flag_suc, flag_fin, flag_inc, flag_min, flag_once1)
      tp = tp1
      bz = bz1
      if flag_suc:
        return rootlist, tdeg, flag_fin, flag_lag
    if flag_suc:
      return  rootlist, tdeg, flag_fin, flag_lag
    flag_inc = True
    while flag_inc:
      flag_inc = False
      bz2 = bz
      tp2 = tp
      tca1 = tca
      rad1 = rad
      print('TO DO minima')
      print(rad)
      res1 = minima(cosa, ads, noco, rad, tdeg, tca)

#Spa

In [16]:
def spa(poly, rootlist):
  tdeg = len(poly)-1
  if tdeg == 0:
    print("The degree of the polynomial must be greater than 0")
    return -1
  if tdeg == 1:
    return [-poly[1]/poly[0]]
  tlim = 0.3
  ads = 0
  bz = 0.0
  diam1 = 1.0 + max(poly[:-1])
  # глобальные флаги, я посмотрю можно ли от каких-нибудь избавиться
  flag_fin = True
  flag_once = True
  flag_once1 = True
  flag_min = False
  flag_lag = False
  flag_mult = False
  qp = []
  #если степень > 2 и все коэф нулевые, кроме первого и последнего (довольно странный частный случай)
  if tdeg > 2 and sum(poly[1:-1] == 0.0):
    flag_suc = True
    # z0 - последний коэф / первый коэф в степени 1 / (степень многочлена)
    z0 = -poly[-1]/ poly[0]
    if z0 > 0.0:
      z0 = np.power(z0 + 0j, 1/tdeg)
    else:
      if tdeg % 2 != 0:
        z0 = -np.power(abs(z0) + 0j, 1/tdeg)
      else:
        z0 = lc(0+1j)*np.power(abs(z0)+0j, 1/tdeg)
    dex = np.cos((2.0/tdeg)*np.pi) + np.sin((2.0/tdeg)*np.pi)*1j
    rootlist, flag_fin, flag_lag = finroot(poly, rootlist, z0, flag_suc, flag_fin)
    for i in range(1, tdeg):
        z0 = dex * z0
        rootlist, flag_fin, flag_lag = finroot(poly, rootlist, z0, flag_suc, flag_fin)
  else:
    dp = np.polyder(poly)
    print('reversed', list(reversed(poly)), list(reversed(dp)))
    tp, gp = P.polydiv(list(reversed(poly)), list(reversed(dp)))
    print(tp, gp)
    if len(gp) == 1 and gp[0] == 0.0:
      flag_mult = True
      tp1 = tp;
      qp = tp;
      tdeg = len(qp)-1
      if tdeg == 1:
        ro = -qp[-1] / qp[0]
        for _ in range(len(poly)-1):
          rootlist.append(ro)
        return rootlist
    else:
      tp = poly
      tp1 = poly
    bzre = lc(0.0 + 0.5j)
    odeg = tdeg
    tdeg = len(tp)-1
    mincoun = 0.0
    bz += bzre;
    tp = shift(bzre, tp1);
    stepfactor = 3
    stepmin = 2
    stepss = 2
    corsteps = 3
    mico = 0
    noco = 12
    sqr3 = np.sqrt(3.0)
    coht = np.array([2., sqr3, 1., 0., -1., -sqr3, -2., -sqr3, -1., 0., 1., sqr3])
    cosa = coht
    while tdeg > 0:
      print('TO DO run')
      print('run tp', tp)
      rootlist, tdeg, flag_fin, flag_lag =  run(cosa, ads, noco, bz, bzre, tp, tp1, tdeg, rootlist, stepfactor, stepmin, flag_fin ,flag_once1)
      print('rootlist', rootlist)
      print('tdeg', tdeg)
  odeg = len(rootlist)
  if not flag_fin:
    tdeg = len(rootlist)
  rl1 = []
  rl2 = []
  rl3 = []
  flag_lag = False
  degp = len(poly)-1
  for i in range(odeg):
    a = rootlist[i]
    if flag_mult:
      b = laguerre(qp, a)
      c = degp * abs(b)
    else:
      b = laguerre(poly, a)
      c = odeg * abs(b)
    b = a - b
    f = 0.0
    while c > eps1:
      if flag_mult:
        d = laguerre(qp, b)
        e = degp * abs(d)
      else:
        d = laguerre(poly, b)
        e = odeg * abs(d)
      d = b - d
      if e < c:
        b = d
        c = e
        f += 1.0
      else:
        break
    rl1.append(b)
    rl2.append(c)
    rl3.append(f)
  print('rl1', rl1, 'rl2', rl2, 'rl3', rl3  )
  dist = np.zeros((odeg, odeg))
  mindist = diam1
  for i in range(odeg-1):
    for j in range(i+1, odeg):
      distt = abs(rl1[i] - rl1[j])
      if distt < mindist:
        mindist = distt
      dist[i, j] = max(0.0, distt-rl2[i]-rl2[j])
      dist[j, i] = dist[i, j]
  for i in range(odeg):
    dist[i][i] = min([dist[i][jj] for jj in range(i)] + [dist[i][jj] for jj in range(i+1, odeg)]) # ????
  for ii in range(odeg):
    rootlist[ii] = [rl1[ii], rl2[ii], dist[ii,ii], rl3[ii]]
  if flag_mult:
    roli = []
    print('TO DO')
  return rootlist

#TEST

In [17]:
pol = np.array([1.0, -5.0, 8.0, -4.0])
roots = []
res = spa(pol, roots)
print('res', res)

reversed [-4.0, 8.0, -5.0, 1.0] [8.0, -10.0, 3.0]
[-0.55555556  0.33333333] [ 0.44444444 -0.22222222]
tp1 [ 1. -5.  8. -4.]
_tp [  3. -10.   8.]
u 0.5j
_tp [  6. -10.]
el (7.25-5j)
den 2.0
_tp [6.]
el (-5+1.5j)
den 6.0
_tp []
el (1+0j)
den 42.0
TO DO run
run tp [(1+0j), (-5+1.5j), (7.25-5j), (-2.75+3.875j)]
trial_tp [(1+0j), (-5+1.5j), (7.25-5j), (-2.75+3.875j)]
tp [(1+0j), (-5+1.5j), (7.25-5j), (-2.75+3.875j)]
1 (7.25-5j)
Test [0.+0.j 0.+0.j 0.+0.j 0.+0.j] 3
q [(1+0j), (-5+1.5j), (7.25-5j), (-2.75+3.875j)]
dq [  3.  +0.j -10.  +3.j   7.25-5.j]
done
kia [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87] 1
tp [(1+0j), (-5+1.5j), (7.25-5j), (-2.75+3.875j)]
rl [2.00000002-0.49999994j 1.99999998-0.50000006j 1.        -0.5j       ]
tqa [0.        +0.j         0.50684932-0.1849315

TypeError: ufunc 'ceil' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
from sympy import symbols, diff, gcd

# Определяем переменную
z = symbols('z')

# Допустим, rp - это ваше выражение
rp = z**2 + 2*z + 1  # Пример выражения

# Вычисляем производную rp по z
dp = diff(rp, z)

print(rp/dp)

# Вычисляем НОД между rp и dp
gp = gcd(rp, dp)
print(f"Функция: {rp}")
print(f"Производная: {dp}")
print(f"НОД: {gp}")

In [ ]:
# Define your polynomials as sequences of coefficients
p = [1, -4, 5, -2]
q = [3, -8, 5, 0]


# Divide p by q
quotient, remainder = P.polydiv(p, q)

print('Quotient:', quotient)
print('Remainder:', remainder)

In [ ]:
test = [1.0]
print(np.polyder(test))

In [ ]:
print([i for i in range(84)])

In [ ]:
myList = [0.0, 0.0, 0.0, 2.0, 2.0]
first_non_zero = next((i for i, x in enumerate(myList) if x), None)
print(first_non_zero)